In [1]:
import pandas as pd
from dash import Dash, dash_table, dcc, html, Input, Output
import dash_bootstrap_components as dbc
import plotly.express as px
import numpy as np

%matplotlib inline

In [2]:
df = pd.read_csv('StudentsPerformance.csv')
df.head()

,gender,race/ethnicity,parental level of education,lunch,test preparation course,math score,reading score,writing score
0,female,group B,bachelor's degree,standard,none,72,72,74
1,female,group C,some college,standard,completed,69,90,88
2,female,group B,master's degree,standard,none,90,95,93
3,male,group A,associate's degree,free/reduced,none,47,57,44
4,male,group C,some college,standard,none,76,78,75


<h1>'Tratamento dos dados'</h1>

In [3]:
df.shape

(1000, 8)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 8 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   gender                       1000 non-null   object
 1   race/ethnicity               1000 non-null   object
 2   parental level of education  1000 non-null   object
 3   lunch                        1000 non-null   object
 4   test preparation course      1000 non-null   object
 5   math score                   1000 non-null   int64 
 6   reading score                1000 non-null   int64 
 7   writing score                1000 non-null   int64 
dtypes: int64(3), object(5)
memory usage: 62.6+ KB


In [5]:
df.describe()

,math score,reading score,writing score
count,1000.00000,1000.000000,1000.000000
mean,66.08900,69.169000,68.054000
std,15.16308,14.600192,15.195657
min,0.00000,17.000000,10.000000
25%,57.00000,59.000000,57.750000
50%,66.00000,70.000000,69.000000
75%,77.00000,79.000000,79.000000
max,100.00000,100.000000,100.000000


In [6]:
df.describe(include=[object])

,gender,race/ethnicity,parental level of education,lunch,test preparation course
count,1000,1000,1000,1000,1000
unique,2,5,6,2,2
top,female,group C,some college,standard,none
freq,518,319,226,645,642


In [7]:
df.isnull().sum()

gender                         0
race/ethnicity                 0
parental level of education    0
lunch                          0
test preparation course        0
math score                     0
reading score                  0
writing score                  0
dtype: int64

In [8]:
print(df['gender'].value_counts())
print('\n')
print(df['race/ethnicity'].value_counts())
print('\n')
print(df['parental level of education'].value_counts())
print('\n')
print(df['lunch'].value_counts())
print('\n')
print(df['test preparation course'].value_counts())

female    518
male      482
Name: gender, dtype: int64


group C    319
group D    262
group B    190
group E    140
group A     89
Name: race/ethnicity, dtype: int64


some college          226
associate's degree    222
high school           196
some high school      179
bachelor's degree     118
master's degree        59
Name: parental level of education, dtype: int64


standard        645
free/reduced    355
Name: lunch, dtype: int64


none         642
completed    358
Name: test preparation course, dtype: int64


<h1>'Dashboard'</h1>

In [9]:
def fig1(value, func='sum'):
    fig1 = px.histogram(
        df, 
        x=value, 
        y=df.describe(include=int).columns, 
        barmode='group', 
        template='seaborn',
        histfunc=func,
        )

    newnames = {'math score': 'Math Score', 'reading score':'Reading Score', 'writing score':'Writing Score'}

    fig1.for_each_trace(
            lambda t: t.update(
                name = newnames[t.name].replace(' ', '<br>'),
                legendgroup = newnames[t.name],
                              )
                          )

    fig1.update_layout(
        legend=dict(tracegroupgap=10),
        bargap=0.4, 
        xaxis_title='', 
        yaxis_title='Total Score',
        plot_bgcolor='rgba(0,0,0,0)',
        paper_bgcolor='rgba(0,0,0,0)',
        hoverlabel=dict(font_size=16),
        legend_title_text='Subjects Scoring:<br>',
        font=dict(size=16),
        title=dict()
        
    )

    fig1.update_traces(hovertemplate='<b>%{x}</b><br><br>Total Score = %{y:.3s}')

    fig1.update_xaxes(zeroline=False, showgrid=False, categoryorder='category ascending')
    fig1.update_yaxes(zeroline=False, showgrid=False)

    return fig1

def fig2(value):
    fig2 = px.pie(
    df,
    names=value,
    template='seaborn',
             )

    fig2.update_traces(
    hovertemplate='<b>%{label}</b><br>Total = %{value}<br>', 
    textinfo='percent'
)

    fig2.update_layout(
    hoverlabel=dict(font_size=16), 
    font=dict(size=16), 
    legend_title_text=value.title(),
    paper_bgcolor='rgba(0,0,0,0)',
    plot_bgcolor='rgba(0,0,0,0)'
)

    return fig2

def fig3(category, func, c, cols, values):
    fig3 = px.histogram(
    df, 
    x=category,  
    y=values,
    barmode='group',
    color=c, 
    facet_col=cols, 
    facet_col_spacing=0.075,
    category_orders={'gender':['male', 'female']},
    color_discrete_map={'male':'darkblue', 'female':'Crimson'},
    histfunc=func,
    template='seaborn'
    )

    fig3.for_each_annotation(lambda a: a.update(text = a.text.split('=')[-1].title()))
    fig3.add_annotation(x=0.5, y=1.15, showarrow=False, text=cols.title(), xref='paper', yref='paper', font=dict(size=20))

    fig3.update_traces(hovertemplate='Total Score = <b>%{y:.3s}</b><extra></extra>')

    fig3.update_xaxes(zeroline=False, showgrid=False, categoryorder='category ascending', title='')
    fig3.update_yaxes(zeroline=False, showgrid=False)

    fig3.update_layout(
        bargap=0.25,
        font=dict(size=16), 
        plot_bgcolor='rgba(0,0,0,0)',
        paper_bgcolor='rgba(0,0,0,0)',
        hoverlabel=dict(font_size=16),
        legend_title_text=c.title(),
        hovermode='x unified',
        yaxis1=dict(title='Total Score')
    )
    return fig3

def fig4(name, value=None):
    fig4 = px.pie(
        df, 
        names=name,
        template='seaborn',
        values=value
    )

    fig4.update_traces(
        hoverlabel=dict(font_size=16), 
        hovertemplate='<b>%{label}</b><br>Total = %{value}',
        textinfo='percent',
        marker=dict(colors=['Crimson', 'darkblue'])
    )

    fig4.update_layout(
        font=dict(size=16), 
        legend_title_text=name.title(),
        plot_bgcolor='rgba(0,0,0,0)',
        paper_bgcolor='rgba(0,0,0,0)'
    )
    return fig4

def fig5(value, var1, var2, color):
    fig5 = px.scatter(
        df, 
        x=var1, 
        y=var2, 
        color=color,
        facet_col=value,
        facet_col_spacing=0.08,
        category_orders={'gender':['male', 'female'], 'race/ethnicity':['group A', 'group B', 'group C', 'group D', 'group E']},
        color_discrete_map={'male':'darkblue', 'female':'Crimson'},
        labels={'male':'Male', 'female':'Female'},
        template='seaborn',
    )
    
    fig5.for_each_annotation(lambda a: a.update(text=a.text.split('=')[-2:][-1]))        
    fig5.add_annotation(x=0.5, y=-1, text=var1.title(), xref='paper', yref='paper', font=dict(size=16))

    fig5.update_traces(
        hoverlabel=dict(font_size=16),
        hovertemplate='<b>Male</b><br><br>x: %{x}<br>y: %{y}<extra></extra>', 
        selector={'name':'male'},
    )

    fig5.update_traces(
        hoverlabel=dict(font_size=16),
        hovertemplate='<b>Female</b><br><br>x: %{x}<br>y: %{y}<extra></extra>', 
        selector={'name':'female'},
        )

    fig5.update_xaxes(zeroline=False, showgrid=False, title='')
    fig5.update_yaxes(zeroline=False, showgrid=False)

    fig5.update_layout(
        font=dict(size=16), 
        legend_title_text=color.title(),
        yaxis1=dict(title=var2.title()),
        plot_bgcolor='rgba(0,0,0,0)',
        paper_bgcolor='rgba(0,0,0,0)'
    )
    return fig5

In [21]:
def update_hist_overview():
    @app.callback(
        Output('fig1', 'figure'),
        Input('drop-1', 'value'),
        Input('radio-1', 'value')
    )
    def update_hist_overview(value, func):
        return fig1(value, func)

def update_pie_overview():
    @app.callback(
        Output('fig2', 'figure'),
        Input('drop-1', 'value')
    )
    def update_pie(value):
        return fig2(value)

def update_histfunc_hist_overview():
    @app.callback(
        Output('fig1', 'figure'),
        Input('radio-1','value'),
        Input('drop-1', 'value')
    )
    def update_histfunc(func, value):
        return fig1(value, func)

def update_hist_detailed():
    @app.callback(
        Output('fig3', 'figure'),
        Input('radio-2','value'),
        Input('drop-1', 'value'),
        Input('drop-2', 'value'),
        Input('drop-3', 'value'),
        Input('drop-4', 'value')
    )
    def update_hist(func, category, c, cols, values):
        if 'all' in values:
            values = df.describe(include=int).columns
            return fig3(category, func, c, cols, values)
        else:
            return fig3(category, func, c, cols, values)

def update_pie_detailed():
    @app.callback(
        Output('fig4', 'figure'),
        Input('drop-5', 'value'),
        Input('drop-6', 'value')
    )
    def update_pie(value, name):
        if 'quantity' in value:
            return fig4(name)
        else:
            return fig4(name, value)

def update_scatter_correlation():
    @app.callback(
        Output('fig5', 'figure'),
        Input('drop-1', 'value'),
        Input('drop-7','value'),
        Input('drop-8', 'value'),
        Input('drop-9', 'value')
    )
    def update_scatter(value, var1, var2, color):
        return fig5(value, var1, var2, color)

def update_subtitle1():
    @app.callback(
        Output('subtitle-1', 'children'),
        Input('drop-1', 'value')
    )
    def update_subtitle(value):
        return 'Proportion for each {}'.format(value)

def update_subtitle2():
    @app.callback(
        Output('subtitle-2', 'children'),
        Input('drop-1', 'value')
    )
    def update_subtitle(value):
        return 'Proportion by {}'.format(value)

In [22]:
def abc():
    return dbc.Row([
        dbc.Col([
            html.H1(children='Student Performance Overview', id='title-1'),
            html.Hr()
        ], style=dict(paddingTop='50px', paddingBottom='50px'))
    ])

In [23]:
items=['sum', 'avg', 'count', 'min', 'max']

In [24]:
def layout():
    return dbc.Container([
    abc(),
    dbc.Row([
        dbc.Col([
            dbc.Card([
                dbc.CardHeader([
                    html.H2('Total Scoring by Subject')
                ]),
                dbc.CardBody([
                    html.P('Choose your category for overview:'),
                    dcc.Dropdown(['race/ethnicity', 'parental level of education'], 'race/ethnicity', id='drop-1'),
                    dcc.RadioItems(items, 'sum', style=dict(marginTop='15px', marginLeft='15px', padding='15px'), inline=True, id='radio-1'),
                    html.Hr(),
                    dcc.Graph(id='fig1')
                ])
            ], color='light')
        ]),
        dbc.Col([
            dbc.Card([
                dbc.CardHeader([
                    html.H2(id='subtitle-1')
                ]),
                dbc.CardBody([
                    dcc.Graph(id='fig2')
                ], className='h-100')
            ], color='light', className='h-100')
        ])
    ]),
    dbc.Row([
        dbc.Col([
            html.H1(children='Detailed View', id='title-2'),
            html.Hr()
        ], style=dict(paddingTop='50px', paddingBottom='50px'))
    ]),
    dbc.Row([
        dbc.Col([
            dbc.Card([
                dbc.CardHeader([
                    html.H2('Total Scoring of Subjects')
                ]),
                dbc.CardBody([
                    dbc.Row([
                        dbc.Col([
                            html.P('Differentiate by:'),
                            dcc.Dropdown(['lunch', 'test preparation course', 'gender'], 'gender', id='drop-2')
                        ]),
                        dbc.Col([
                            html.P('Split Graph into:'),
                            dcc.Dropdown(['lunch', 'test preparation course', 'gender'], 'lunch', id='drop-3')
                        ]),
                        dbc.Col([
                            html.P('Total Scoring in:'),
                            dcc.Dropdown(list(df.describe(include=int).columns)+['all subjects'], 'all subjects', id='drop-4')
                        ]),
                        dcc.RadioItems(items, 'sum', style={'font-size':'20', 'marginTop':'15px'}, inline=True, id='radio-2')
                    ]),
                    html.Hr(),
                    dcc.Graph(id='fig3')
                ])
            ], color='light')
        ]),
        dbc.Col([
            dbc.Card([
                dbc.CardHeader([
                    html.H2(id='subtitle-2')
                ]),
                dbc.CardBody([
                    html.Div([
                        html.P('Wich values use in the proportion?'),
                        dcc.Dropdown(list(df.describe(include=int).columns)+['quantity of each category'], 'quantity of each category', id='drop-5'),
                    ], style=dict(width='48%', display='inline-block')),
                    html.Div([
                        html.P('choose differentation:'),
                        dcc.Dropdown(['gender', 'test preparation course', 'lunch'], 'gender', id='drop-6'),
                    ], style=dict(width='48%', display='inline-block', marginLeft='15px')),
                    html.Hr(),
                    dcc.Graph(id='fig4')
                ])
            ], color='light', className='h-100')
        ])
    ]),
    dbc.Row([
        dbc.Col([
            html.H1('Correlation'),
            html.Hr(),
        ], style=dict(paddingTop='50px', paddingBottom='50px'))
    ]),
    dbc.Row([
        dbc.Col([
            dbc.Card([
                dbc.CardBody([
                    html.Div([
                        html.P('Choose one variable:'),
                        dcc.Dropdown(df.describe(include=int).columns, 'reading score', id='drop-7')
                    ], style=dict(
                        float='left', 
                        width='32%',
                        display='inline-block',
                        marginRight='15px'
                    )),
                    html.Div([
                        html.P('Choose one variable:'),
                        dcc.Dropdown(df.describe(include=int).columns, 'math score', id='drop-8')
                    ], style=dict(
                        width='32%', 
                        display='inline-block',
                        marginLeft='15px'
                    )),
                    html.Div([
                        html.P('choose differentation:'),
                        dcc.Dropdown(['gender', 'test preparation course', 'lunch'], 'gender', id='drop-9')
                    ], style=dict(marginLeft='15px', width='31%', display='inline-block')),
                    html.Hr(),
                    dcc.Graph(id='fig5')
                ])
            ], style=dict(marginBottom='50px'), color='light')
        ])
    ])
])

In [14]:
layout = dbc.Container([
    abc(),
    dbc.Row([
        dbc.Col([
            dbc.Card([
                dbc.CardHeader([
                    html.H2('Total Scoring by Subject')
                ]),
                dbc.CardBody([
                    html.P('Choose your category for overview:'),
                    dcc.Dropdown(['race/ethnicity', 'parental level of education'], 'race/ethnicity', id='drop-1'),
                    dcc.RadioItems(items, 'sum', style=dict(marginTop='15px', marginLeft='15px', padding='15px'), inline=True, id='radio-1'),
                    html.Hr(),
                    dcc.Graph(id='fig1')
                ])
            ], color='light')
        ]),
        dbc.Col([
            dbc.Card([
                dbc.CardHeader([
                    html.H2(id='subtitle-1')
                ]),
                dbc.CardBody([
                    dcc.Graph(id='fig2')
                ], className='h-100')
            ], color='light', className='h-100')
        ])
    ]),
    dbc.Row([
        dbc.Col([
            html.H1(children='Detailed View', id='title-2'),
            html.Hr()
        ], style=dict(paddingTop='50px', paddingBottom='50px'))
    ]),
    dbc.Row([
        dbc.Col([
            dbc.Card([
                dbc.CardHeader([
                    html.H2('Total Scoring of Subjects')
                ]),
                dbc.CardBody([
                    dbc.Row([
                        dbc.Col([
                            html.P('Differentiate by:'),
                            dcc.Dropdown(['lunch', 'test preparation course', 'gender'], 'gender', id='drop-2')
                        ]),
                        dbc.Col([
                            html.P('Split Graph into:'),
                            dcc.Dropdown(['lunch', 'test preparation course', 'gender'], 'lunch', id='drop-3')
                        ]),
                        dbc.Col([
                            html.P('Total Scoring in:'),
                            dcc.Dropdown(list(df.describe(include=int).columns)+['all subjects'], 'all subjects', id='drop-4')
                        ]),
                        dcc.RadioItems(items, 'sum', style={'font-size':'20', 'marginTop':'15px'}, inline=True, id='radio-2')
                    ]),
                    html.Hr(),
                    dcc.Graph(id='fig3')
                ])
            ], color='light')
        ]),
        dbc.Col([
            dbc.Card([
                dbc.CardHeader([
                    html.H2(id='subtitle-2')
                ]),
                dbc.CardBody([
                    html.Div([
                        html.P('Wich values use in the proportion?'),
                        dcc.Dropdown(list(df.describe(include=int).columns)+['quantity of each category'], 'quantity of each category', id='drop-5'),
                    ], style=dict(width='48%', display='inline-block')),
                    html.Div([
                        html.P('choose differentation:'),
                        dcc.Dropdown(['gender', 'test preparation course', 'lunch'], 'gender', id='drop-6'),
                    ], style=dict(width='48%', display='inline-block', marginLeft='15px')),
                    html.Hr(),
                    dcc.Graph(id='fig4')
                ])
            ], color='light', className='h-100')
        ])
    ]),
    dbc.Row([
        dbc.Col([
            html.H1('Correlation'),
            html.Hr(),
        ], style=dict(paddingTop='50px', paddingBottom='50px'))
    ]),
    dbc.Row([
        dbc.Col([
            dbc.Card([
                dbc.CardBody([
                    html.Div([
                        html.P('Choose one variable:'),
                        dcc.Dropdown(df.describe(include=int).columns, 'reading score', id='drop-7')
                    ], style=dict(
                        float='left', 
                        width='32%',
                        display='inline-block',
                        marginRight='15px'
                    )),
                    html.Div([
                        html.P('Choose one variable:'),
                        dcc.Dropdown(df.describe(include=int).columns, 'math score', id='drop-8')
                    ], style=dict(
                        width='32%', 
                        display='inline-block',
                        marginLeft='15px'
                    )),
                    html.Div([
                        html.P('choose differentation:'),
                        dcc.Dropdown(['gender', 'test preparation course', 'lunch'], 'gender', id='drop-9')
                    ], style=dict(marginLeft='15px', width='31%', display='inline-block')),
                    html.Hr(),
                    dcc.Graph(id='fig5')
                ])
            ], style=dict(marginBottom='50px'), color='light')
        ])
    ])
])

In [25]:
app = Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

app.layout = layout()

update_hist_overview()
update_pie_overview()
update_hist_detailed()
update_pie_detailed()
update_scatter_correlation()
update_subtitle1()
update_subtitle2()


In [ ]:
if __name__ == '__main__':
  app.run_server(debug=False, use_reloader=False)

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [04/Jul/2022 21:51:10] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [04/Jul/2022 21:51:11] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [04/Jul/2022 21:51:11] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [04/Jul/2022 21:51:11] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 200 -
127.0.0.1 - - [04/Jul/2022 21:51:11] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 200 -
127.0.0.1 - - [04/Jul/2022 21:51:11] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 200 -
127.0.0.1 - - [04/Jul/2022 21:51:11] "POST /_dash-update-component HTTP/1.1" 200 -


Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "/sysroot/home/roamanogameplay/.local/lib/python3.9/site-packages/flask/app.py", line 2077, in wsgi_app
    response = self.full_dispatch_request()
  File "/sysroot/home/roamanogameplay/.local/lib/python3.9/site-packages/flask/app.py", line 1525, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/sysroot/home/roamanogameplay/.local/lib/python3.9/site-packages/flask/app.py", line 1523, in full_dispatch_request
    rv = self.dispatch_request()
  File "/sysroot/home/roamanogameplay/.local/lib/python3.9/site-packages/flask/app.py", line 1509, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**req.view_args)
  File "/sysroot/home/roamanogameplay/.local/lib/python3.9/site-packages/dash/dash.py", line 1372, in dispatch
    response.set_data(func(*args, outputs_list=outputs_list))
  File "/sysroot/home/roamanogameplay/.local/lib/python3.9/site-packages/

127.0.0.1 - - [04/Jul/2022 21:51:11] "POST /_dash-update-component HTTP/1.1" 500 -
127.0.0.1 - - [04/Jul/2022 21:51:11] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [04/Jul/2022 21:51:11] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [04/Jul/2022 21:51:11] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [04/Jul/2022 21:51:11] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [04/Jul/2022 21:51:11] "POST /_dash-update-component HTTP/1.1" 200 -


Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "/sysroot/home/roamanogameplay/.local/lib/python3.9/site-packages/flask/app.py", line 2077, in wsgi_app
    response = self.full_dispatch_request()
  File "/sysroot/home/roamanogameplay/.local/lib/python3.9/site-packages/flask/app.py", line 1525, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/sysroot/home/roamanogameplay/.local/lib/python3.9/site-packages/flask/app.py", line 1523, in full_dispatch_request
    rv = self.dispatch_request()
  File "/sysroot/home/roamanogameplay/.local/lib/python3.9/site-packages/flask/app.py", line 1509, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**req.view_args)
  File "/sysroot/home/roamanogameplay/.local/lib/python3.9/site-packages/dash/dash.py", line 1372, in dispatch
    response.set_data(func(*args, outputs_list=outputs_list))
  File "/sysroot/home/roamanogameplay/.local/lib/python3.9/site-packages/

...

127.0.0.1 - - [04/Jul/2022 21:51:19] "POST /_dash-update-component HTTP/1.1" 200 -


Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "/sysroot/home/roamanogameplay/.local/lib/python3.9/site-packages/flask/app.py", line 2077, in wsgi_app
    response = self.full_dispatch_request()
  File "/sysroot/home/roamanogameplay/.local/lib/python3.9/site-packages/flask/app.py", line 1525, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/sysroot/home/roamanogameplay/.local/lib/python3.9/site-packages/flask/app.py", line 1523, in full_dispatch_request
    rv = self.dispatch_request()
  File "/sysroot/home/roamanogameplay/.local/lib/python3.9/site-packages/flask/app.py", line 1509, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**req.view_args)
  File "/sysroot/home/roamanogameplay/.local/lib/python3.9/site-packages/dash/dash.py", line 1372, in dispatch
    response.set_data(func(*args, outputs_list=outputs_list))
  File "/sysroot/home/roamanogameplay/.local/lib/python3.9/site-packages/

127.0.0.1 - - [04/Jul/2022 21:51:19] "POST /_dash-update-component HTTP/1.1" 200 -


Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "/sysroot/home/roamanogameplay/.local/lib/python3.9/site-packages/flask/app.py", line 2077, in wsgi_app
    response = self.full_dispatch_request()
  File "/sysroot/home/roamanogameplay/.local/lib/python3.9/site-packages/flask/app.py", line 1525, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/sysroot/home/roamanogameplay/.local/lib/python3.9/site-packages/flask/app.py", line 1523, in full_dispatch_request
    rv = self.dispatch_request()
  File "/sysroot/home/roamanogameplay/.local/lib/python3.9/site-packages/flask/app.py", line 1509, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**req.view_args)
  File "/sysroot/home/roamanogameplay/.local/lib/python3.9/site-packages/dash/dash.py", line 1372, in dispatch
    response.set_data(func(*args, outputs_list=outputs_list))
  File "/sysroot/home/roamanogameplay/.local/lib/python3.9/site-packages/

127.0.0.1 - - [04/Jul/2022 21:51:19] "POST /_dash-update-component HTTP/1.1" 500 -
127.0.0.1 - - [04/Jul/2022 21:51:19] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [04/Jul/2022 21:51:19] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [04/Jul/2022 21:51:19] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [04/Jul/2022 21:51:33] "GET /_dash-component-suites/dash/dcc/dash_core_components-shared.js.map HTTP/1.1" 200 -
127.0.0.1 - - [04/Jul/2022 21:51:33] "GET /_dash-component-suites/dash/dash_table/bundle.js.map HTTP/1.1" 200 -
127.0.0.1 - - [04/Jul/2022 21:51:33] "GET /_dash-component-suites/dash/html/dash_html_components.min.js.map HTTP/1.1" 200 -
127.0.0.1 - - [04/Jul/2022 21:51:33] "GET /_dash-component-suites/dash/dcc/dash_core_components.js.map HTTP/1.1" 200 -
127.0.0.1 - - [04/Jul/2022 21:51:33] "GET /_dash-component-suites/dash/dcc/async-dropdown.js.map HTTP/1.1" 200 -
127.0.0.1 - - [04/Jul/2022 21:51:33] "GET /_dash-component-suites/dash/dcc